<a href="https://colab.research.google.com/github/jbrown544/interlingual-topic-modeling/blob/main/Interlingual_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Thematic Discovery in Non-English Natural Language Corpora

**Use the "Open in Colab" link above to render visualizations.*

## Processing Environment Initialization

[![Built with spaCy](https://img.shields.io/badge/built%20with-spaCy-09a3d5.svg)](https://spacy.io)

In [1]:
# supress library maintenance warnings

import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
# warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [2]:
# update package management then install required versions of dependencies
!pip install -q -U pip setuptools wheel python-Levenshtein numpy \
                   gensim==4.1.0 spacy==3.1.2 pyLDAvis==3.3.1

In [3]:
# inspect Gensim package information
!pip show gensim

Name: gensim
Version: 4.1.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, smart-open, scipy
Required-by: pyLDAvis


In [4]:
# inspect spaCy package information
!pip show spacy

Name: spacy
Version: 3.1.2
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: srsly, setuptools, pydantic, pathy, catalogue, requests, typer, wasabi, numpy, thinc, blis, preshed, jinja2, murmurhash, typing-extensions, cymem, spacy-legacy, tqdm, packaging
Required-by: fastai, en-core-web-sm


In [5]:
# instpect pyLDAvis package information
!pip show pyLDAvis

Name: pyLDAvis
Version: 3.3.1
Summary: Interactive topic model visualization. Port of the R package.
Home-page: https://github.com/bmabey/pyLDAvis
Author: Ben Mabey
Author-email: ben@benmabey.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: funcy, joblib, scikit-learn, future, gensim, setuptools, numexpr, sklearn, jinja2, scipy, numpy, pandas
Required-by: 


## Data Environment Initialization

In [6]:
# import dependencies

from urllib.parse import urljoin

from gensim import models
from gensim import corpora
from gensim.utils import simple_preprocess

from smart_open import open

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X

In [7]:
# enable visualizations inline
pyLDAvis.enable_notebook()

### Language-Neutral Processing Pipelines

In [8]:
# basic text preprocessing
def basic_preprocess(texts,
                     deaccent=False,
                     min_token_length=2,
                     max_token_length=15):
  
  # tokenizization, lowercasing, filtering tokens to length range,
  # and optionally removing accenting marks
  return [simple_preprocess(text, deacc=deaccent,
                             min_len=min_token_length,
                             max_len=max_token_length)
  for text in texts]

In [9]:
# basic text preprocessing
def advanced_preprocess(texts, language):
  pass

### French Language Samples

In [10]:
# samples from the French language

# samples are accessible from within this repository
french_book_url = ('https://raw.githubusercontent.com/jbrown544/'
                   'interlingual-topic-modeling/main/text/books/fr/')

jules_verne_french_books = [
  urljoin(french_book_url, 'Around_the_World_in_Eighty_Days.txt'),
  urljoin(french_book_url, 'Five_Weeks_in_a_Balloon.txt'),
  urljoin(french_book_url, 'From_the_Earth_to_the_Moon.txt'),
  urljoin(french_book_url, 'Robur_the_Conqueror.txt'),
  urljoin(french_book_url, 'The_Mutineers_of_the_Bounty.txt'),
  urljoin(french_book_url, 'The_Begums_Fortune.txt'),
  urljoin(french_book_url, 'The_Blockade_Runners.txt')
]

#### Process French Language Samples

In [11]:
# read and process documents into processed docs

encoding = 'latin-1'
jules_verne_french_texts = [open(book, encoding=encoding).read() \
                            for book in jules_verne_french_books]

jules_verne_docs = basic_preprocess(jules_verne_french_texts)

In [12]:
# construct dictionary from processed docs
jules_verne_dictionary = corpora.Dictionary(jules_verne_docs)

print(jules_verne_dictionary)

Dictionary(23777 unique tokens: ['abaissait', 'abaissant', 'abaissement', 'abaissé', 'abandonna']...)


In [13]:
# construct corpus (term-document matrix) by converting 
# processed lists into bag-of-words representation
jules_verne_corpus = [jules_verne_dictionary.doc2bow(doc) \
                      for doc in jules_verne_docs]

## HDP Topic Modelling Inspection (Automatic Case)

In [14]:
# construct HDP model (topics discovered during processing)
jules_verne_hdp_model = models.HdpModel(jules_verne_corpus, 
                                        id2word=jules_verne_dictionary)

In [15]:
# visualize suggested LDA model derived from computed HDP model
gensimvis.prepare(jules_verne_hdp_model.suggested_lda_model(),
                  jules_verne_corpus, jules_verne_dictionary)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster          Freq
topic                                                   
0     -0.000163 -0.000004       1        1  4.081319e+01
1     -0.000059 -0.000165       2        1  2.141536e+01
2     -0.000050 -0.000016       3        1  2.040938e+01
3     -0.000028  0.000117       4        1  1.528584e+01
4     -0.000224 -0.000436       5        1  2.076155e+00
...         ...       ...     ...      ...           ...
145   -0.000229 -0.000072     146        1  2.905058e-47
146   -0.000010 -0.000180     147        1  1.452529e-47
147   -0.000138 -0.000047     148        1  7.262644e-48
148    0.000100  0.000149     149        1  3.631322e-48
149   -0.000200  0.000015     150        1  3.631322e-48

[150 rows x 5 columns], topic_info=               Term          Freq      Total  Category  logprob  loglift
10745     diversité  1.700000e+01  17.000000   Default  30.0000  30.0000
6369        preuves  1.700000e+01  17.000000   Default  29.0000  29.0000
3688   fuligineuses  1.600000e+01  16.000000   Default  28.0000  28.0000
19064         glass  1.700000e+01  17.000000   Default  27.0000  27.0000
5473          noire  1.600000e+01  16.000000   Default  26.0000  26.0000
...             ...           ...        ...       ...      ...      ...
17733   travaillent  6.878729e-49  14.193416  Topic150  -9.7840   0.2886
11115     encombrée  6.880005e-49  14.386846  Topic150  -9.7839   0.2753
21735      employez  6.909264e-49  14.730084  Topic150  -9.7796   0.2560
21695     détruirai  6.876988e-49  14.599670  Topic150  -9.7843   0.2602
5247         monter  6.844856e-49  14.788531  Topic150  -9.7890   0.2426

[7883 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1         1  0.422606    abaissant
1         2  0.211303    abaissant
1         3  0.211303    abaissant
1         4  0.140869    abaissant
2         1  0.445198  abaissement
...     ...       ...          ...
9358      4  0.137785          île
9361      1  0.432792        îlots
9361      2  0.247310        îlots
9361      3  0.185482        îlots
9361      4  0.123655        îlots

[23264 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150])

## Translation Environment Initialization

In [16]:
# set environment variable to point to your own service account key
%env GOOGLE_APPLICATION_CREDENTIALS /content/keys/interlingual-topic-modeling.json

env: GOOGLE_APPLICATION_CREDENTIALS=/content/keys/interlingual-topic-modeling.json


In [17]:
# sample code from google documentation
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    results = translate_client.translate(text, target_language=target)

    for result in results:
      print(u"Text: {}".format(result["input"]))
      print(u"Translation: {}".format(result["translatedText"]))
      print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

In [18]:
# test translatoins
translate_text('fr', ['pencil', 'pen', 'paper'])

Text: pencil
Translation: crayon
Detected source language: en
Text: pen
Translation: stylo
Detected source language: en
Text: paper
Translation: papier
Detected source language: en
